In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import glob #glob is a python version of running the UNIX ls command. Comes in handy for finding files
import pandas as pd
from astropy.io import fits
from astropy.table import Table
from astropy.coordinates import SkyCoord
from astropy import units as u
from astroquery.cadc import Cadc #This is the import of specifc archive of Astroquery

In [ ]:
# Input: RA, DEC, Radius, FITS name from catalog
# Code will generate a PDF of spectra and HST cutouts from a list of coordinates
# Output: PDF with 5 pairs of spectra & images plotted per page

In [ ]:
def get_images_from_list(image_list, show_progress=False):
    """Returns lists of HDUs from given image list URLS"""
    from astroquery.utils import commons
    fits_list = []
    fn = [commons.FileContainer(url, encoding='binary',show_progress=show_progress) for url in image_list]
    for f in fn:
        try:
            fits_list.append(f.get_fits())
        except:
            pass
    return fits_list

In [ ]:
# Find the object's spectrum from coordinates
def find_obj_spectrum(fitsname):
    """Finds the object's spectrum from it's coordinates.
    Takes in the FITS name from catalog"""
    
    cat_tab = Table.read(fitsname, format='fits')
    
    
    secondary = cat_tab['f_primary'] != 1 
    good_z = cat_tab['f_z'] == 0
    good_sn = cat_tab['SN'] >= 5
#     good_spec = cat_tab['f_spec'] == 0
    mask = np.all([secondary, good_z, good_sn], axis=0) 
    print("%d objects found."%(mask.sum()))
 
    
    
    # Grabbing the coordinates of the objects of interest
    samp_filenames = cat_tab['Filename'][mask]
    samp_ra = cat_tab['RAJ2000'][mask]
    samp_dec = cat_tab['DECJ2000'][mask]
    samp_specid = cat_tab['SPECT_ID'][mask]
    samp_z = cat_tab['z'][mask] # collecting redshift data
    samp_SN = cat_tab['SN'][mask]
    
    #Dictionary of each fits file we found, with its corresponding ra and dec, spec_id, redshift (z) as the values
    object_info = {filename:[ra,dec, spec_id, z, SN] for filename, ra, dec, spec_id, z, SN in zip(samp_filenames, samp_ra, samp_dec, samp_specid, samp_z, samp_SN)}
    
    
#     object_coordinates = list(files_and_coords.values()) # list of list of ra&dec
    return object_info

#### We will want to loop over to plot each image for the files in samp_filenames ...
#### So we want to loop over all the coords in samp_ra and samp_dec lists

#### samp_filenames[0]  ---> samp_ra[0]  -----> samp_dec[0]
    
obj_coordinates = find_obj_spectrum('./legac_dr2_cat.fits')

In [ ]:
# Grab HST cutout from coordinates

# Loops through coordinates inside thte function but maybe change that 
def get_HST_image_cutouts(object_info, radius=5.5*u.arcsecond):
    """Grab HST image cutouts from object's coordinates."""

    object_coordinates = list(object_info.values()) # object_coordinates is now a list of list of ra&dec
    
    #Query HST Database
    
    total_fits_list = []
    
    #ITERATING THRU ALL COORDINATES
    for ra_and_dec in object_coordinates:  # iterates thru each pair of coordinates for each fits
        ra = ra_and_dec[0] # first item in each coordinates_list is the ra
        dec = ra_and_dec[1] # second item in each coordinates_list is the dec
        
        #Create SkyCoord object using Astropy and the object's coordinates
        coords = SkyCoord(ra=ra*u.degree,dec=dec*u.degree)

        #Create a CADC object and query it using query region (coordinates and cutout around our object) for the HST collection
        cadc = Cadc()
        result = cadc.query_region(coords,radius=radius,collection='HST')

        #Select entries above calibration level 2 and for the ACS instrument
        result_cal = result[np.logical_and(result['calibrationLevel'] > 2,result['instrument_name'] == 'ACS/WFC')]
        #Grabs composite images
        result_comp = result_cal[result_cal['typeCode']=='C']

        #Take results and find image list
        image_list = cadc.get_image_list(result_comp,coordinates=coords,radius=radius)
    #     print(image_list)

        #Grab HDUlists from image_list (which is a list of URLs)
        fits_list = get_images_from_list(image_list) # list of HDUlists
        
        #Create dummy image to add to fits list  
        #So we wont get an error later if no images were found for some fits files
        dummy_image = np.zeros((255,256)) 
        
        if len(fits_list) == 0:
            fits_list.append([dummy_image])
        
        for fits in fits_list:
            if len(fits) == 0:
                fits.append(dummy_image)            
        
        total_fits_list.append(fits_list)
        
#         try:
        # if fits_list was not empty, try to get the photname / units / other info from the header

#         except:
        # except if the fits_list is empty or if it couldnt find the ['SCI'] extension / etc,
        # (which means the image is the dummy image), then just plot the np.zeros dummy image
    
    
    return total_fits_list
    
fits_list_HST = get_HST_image_cutouts(obj_coordinates)
# print(len(fits_list_HST), fits_list_HST)

In [ ]:
# for fit in fits_list_HST:
#     print((fit))
for fit in fits_list_HST[0]:
    print(fit)

In [ ]:
def trim_axs(axs, N):
        """
        Reduce *axs* to *N* Axes. All further Axes are removed from the figure. 
        (removes an axis so that way there wont be a plot at that entry.....)
        """
        axs = axs.flat
        for ax in axs[N:]:
            ax.remove()

        return axs[:N]

In [ ]:
# need to add code to print on diff pdf pages
# pp = PdfPages('multipage_test.pdf')
from matplotlib.backends.backend_pdf import PdfPages

def plot_spectra_and_image(fits_list, object_info):
    """ 
    Plots spectrum and image cutout for each item in fits_list. Saves plots in a PDF.
    fits_list is the list of HDUlists returned from get_image_list urls
    object_info is the dictionary with form {filename:[ra,dec, spec_id, z]} created in the
    first function find_object_spectrum
    """
    
    from matplotlib import cm, colors
    pp = PdfPages('multipage_test_PLOTS.pdf') 

    spectra_filenames = list((object_info.keys()))
    spectra_info = list(object_info.values())
    
    
    total_plot_counter = 0
    local_plot_counter = 0 #Keeps count of which plot (0-4) on the pdf page its currently on
    num_of_plots = len(fits_list) #Total number of plots we will make = how many objs there are in fits_list
    plots_per_page = 5 #We want 5 pair plots in each pdf page
    tot_num_of_pages = int(np.ceil(num_of_plots / float(plots_per_page)))
    
    page_counter = 1
    
    
    howmanylefttoplot = num_of_plots - total_plot_counter

    xdim = plots_per_page*2.5 
    ydim = plots_per_page*5
    
    print('Need total of ', tot_num_of_pages, ' pages\n')
    
    fig, axs = plt.subplots(plots_per_page, 2, figsize=(xdim,ydim))

    
    for fits_ob, file, spec_info in zip(fits_list, spectra_filenames,spectra_info):
        
        spec_id = spec_info[2]
        spec_redshift = spec_info[3]
        spec_SN = spec_info[4]

        #         try:
#             This conditional helps ignore some returned composite images that have nans
#             if np.isnan(fits['SCI'].data).sum() < 1:
        
#                         fig, ax = plt.subplots(figsize=(10,10))
#         fits_obj = fits_ob[0]

        # SPECTRA
        obj0 = fits.open('./spectraDR/'+file.strip())
        wave = obj0[1].data['WAVE'][0]
        flux = obj0[1].data['FLUX'][0]
        #Masking to plot only nonzero values of flux (to just look at the interesting parts of the spectra)
        flux_cutoff = np.where(flux != 0)
        flux = flux[flux_cutoff]
        #Plot wave only where flux != 0
        wave = wave[flux_cutoff]

        spec_info_annotate = 'Spec_ID: ' + str(spec_id) + '\nRedshift (z): ' + str(spec_redshift) + '\nS/N: ' + str(spec_SN)
#         font = {'family': 'serif',
#         'color':  'darkred',
#         'weight': 'normal',
#         'size': 12,
#         }        
        
        
        
        print('FLUX ', len(flux))
        #IMAGE
        try:
            photname = fits_ob[0]['SCI'].header['PHOTMODE']
            units = fits_ob[0]['SCI'].header['BUNIT']
        except:
#             try: #try looking at a diff image to see if its better?
#                 photname = fits_ob[1]['SCI'].header['PHOTMODE']
#                 units = fits_ob[1]['SCI'].header['BUNIT']
#             except:
    #         except if the ['SCI'].data caused an error/couldnt be found because the image is an np.array of zeros ~ dummy image
            photname = 'No image found'
            units = 'None'


            
        #If we havent reached the 5th plot of the page yet, keep adding plots to the same fig
        #Note: local_plot_counter goes from 0 --> 4 but plots_per_page == 5 so need strictly <
        if local_plot_counter < plots_per_page and page_counter < tot_num_of_pages:
            #Plot on 'existing' fig/page  
            #Plotting spectra
            axs[local_plot_counter, 0].plot(wave, flux, '-')
            axs[local_plot_counter, 0].set_xlim(wave[0],wave[-2])
            axs[local_plot_counter, 0].set_title(file.strip('.fits').strip('fits'))
            axs[local_plot_counter, 0].set_xlabel('Wave (A)')
            axs[local_plot_counter, 0].set_ylabel('Flux')
#             axs[local_plot_counter, 0].annotate(spec_info_annotate, (wave[220],flux[-15]),bbox={'alpha':0.1, 'pad':10})
#             plt.figtext(0.5, 0, spec_info_annotate, ha="center", fontsize=7, bbox={"alpha":0.1, "pad":5})
            axs[local_plot_counter, 0].annotate(spec_info_annotate, xy = (30, 275), xycoords='axes pixels', va='top',ha='left',bbox={'alpha':0.1, 'pad':10})

 

            #Plotting image
            try:
                axs[local_plot_counter, 1].imshow(fits_ob[0]['SCI'].data)
    #             print(fits_ob[0]['SCI'].data.shape)
               

            except: # if no image was found ---> plot empty image 
    #             try: # try to print a different image if the first one [0th] wasn't working?
    #                 axs[local_plot_counter, 1].imshow(fits_ob[1]['SCI'].data)
    # #                 print(fits_ob[0]['SCI'].data.shape)
    #             except:
#                 axs[local_plot_counter, 1].imshow(np.zeros((279,279)))
                axs[local_plot_counter, 1].imshow(np.zeros((279,279)))
#                 cbar = fig.colorbar(HST_image)
#                 cbar.set_label(units)

            #Title & Label for image
            axs[local_plot_counter, 1].set_title(photname)
            axs[local_plot_counter, 1].set_ylabel(units)

            
            #Annotate plot: redshift, spec_id
#             cbar = fig.colorbar(HST_image)
#             cbar.set_label(units)
            
            plt.tight_layout()
#             cbar = fig.colorbar()
#             axs.set_title(photname)
#             cbar.set_label(units)
            local_plot_counter +=1
            total_plot_counter +=1

        
            print(axs.shape)
            
            
        #If local_plot_counter has reached/surpassed the max number of plots per page, then
        #Save the previously plotted figs onto a pdf page, create a new subplots fig, 
        #Reset local_plot_counter = 0 and then plot the current objects onto the new fig
        elif local_plot_counter >= plots_per_page or page_counter == tot_num_of_pages:
            #Save the first 5 figures on a pdf page before we create a new fig for plotting the next plots on the next pdf page
            
            
            if page_counter < tot_num_of_pages:
                page_counter+=1
                local_plot_counter = 0
                pp.savefig()
            
            

            #If we are on the final page:
            if page_counter == tot_num_of_pages:
                #If we need to plot less than 5 plots:
                #Create fig with only the number of plots we have left, so we dont have extra empty axes on the page
                howmanylefttoplot = num_of_plots - total_plot_counter
#                 local_plot_counter = 0

                
                
                if local_plot_counter == 0: 
                    fig, axs = plt.subplots(5, 2, figsize=(xdim,ydim))
                    print(axs.shape)
                    axs = trim_axs(axs, howmanylefttoplot*2)
                    print(axs.shape)
                    axs = axs.reshape(-1,2)
                    print(axs.shape)

                    plt.tight_layout()
                
                
            #If we aren't on the final page:
            #Create fig that plots 5 pairs of spectra&images
            else:
                #Create new fig:  
                fig, axs = plt.subplots(5, 2, figsize=(xdim,ydim))

#             fig, axs = plt.subplots(min(plots_per_page,howmanylefttoplot), 2, figsize=(xdim,ydim))

            #Reset local_plot_counter
#             local_plot_counter = 0
            
            #Plot spectra as first obj in new fig
            axs[local_plot_counter, 0].plot(wave, flux, '-')
            axs[local_plot_counter, 0].set_xlim(wave[0],wave[-1])
            axs[local_plot_counter, 0].set_title(file.strip('.fits').strip('fits'))
            axs[local_plot_counter, 0].set_xlabel('Wave (A)')
            axs[local_plot_counter, 0].set_ylabel('Flux')
#           
            #Plot image
            try:
#                 axs[local_plot_counter, 1].imshow(fits_ob[0]['SCI'].data)
#                 HST_image = axs[local_plot_counter, 1].imshow(fits_ob[0]['SCI'].data
                axs[local_plot_counter, 1].imshow(fits_ob[0]['SCI'].data)
               
        
            except: # if no image was found ---> plot empty image 
#                 axs[local_plot_counter, 1].imshow(np.zeros((279,279)))
                axs[local_plot_counter, 1].imshow(np.zeros((279,279)))
#                 cbar = fig.colorbar(HST_image)
#                 cbar.set_label(units)


            axs[local_plot_counter, 1].set_title(photname)
            axs[local_plot_counter, 1].set_ylabel(units)
#             plt.figtext(0.5, 0, spec_info_annotate, ha="center", fontsize=7, bbox={"alpha":0.1, "pad":5})
#             axs[local_plot_counter, 0].annotate(spec_info_annotate, (wave[320],flux[320]),bbox={'alpha':0.1, 'pad':10})
            axs[local_plot_counter, 0].annotate(spec_info_annotate, xy = (30, 275), xycoords='axes pixels', va='top',ha='left',bbox={'alpha':0.1, 'pad':10})

             
            plt.tight_layout()
            local_plot_counter +=1
            total_plot_counter +=1
            
            
        print('Spectra: ',file.strip('.fits').strip('fits'), '\nImage: ',photname,  '\nLocal Plot Counter: ', local_plot_counter, '\nTotal Plot Counter: ', total_plot_counter)
#         print('Spect_ID: ', spec_id, '\nRedshift: z = ', z_redshift, '\n')
    pp.savefig()

    pp.close()


In [ ]:
plot_spectra_and_image(fits_list_HST, obj_coordinates)

### UPDATE to remove empty axes from last pdf page